In [1]:
# import the necessary packages
# This is working 
import numpy as np
import imutils
import cv2

class Stitcher:
	def __init__(self,detector):
		# determine if we are using OpenCV v3.X
		self.isv3 = imutils.is_cv3(or_better=True)
		self.detector=detector
		self.counter=1
	def stitch(self, images, ratio=0.75, reprojThresh=4.0,
		showMatches=False):
		# unpack the images, then detect keypoints and extract
		# local invariant descriptors from them
		(imageB, imageA) = images
		(kpsA, featuresA) = self.detectAndDescribe(imageA)
		(kpsB, featuresB) = self.detectAndDescribe(imageB)
		

		# match features between the two images
		M = self.matchKeypoints(kpsA, kpsB,
			featuresA, featuresB, ratio, reprojThresh)
		# if the match is None, then there aren't enough matched
		# keypoints to create a panorama
		if M is None:
			return None
		# otherwise, apply a perspective warp to stitch the images
		# together
		(matches, H, status) = M
		result = cv2.warpPerspective(imageA, H,
			(imageA.shape[1] + imageB.shape[1], imageA.shape[0]))
		result[0:imageB.shape[0], 0:imageB.shape[1]] = imageB
		print(f"For counter = {self.counter} and detector = {self.detector} the matchpoint number is {len(matches)}")
		self.counter +=1
		# check to see if the keypoint matches should be visualized
		if showMatches:
			vis = self.drawMatches(imageA, imageB, kpsA, kpsB, matches,
				status)
			# return a tuple of the stitched image and the
			# visualization
			return (result, vis)
		# return the stitched image
		return result		
	def detectAndDescribe(self, image):
		# convert the image to grayscale
		gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
		# check to see if we are using OpenCV 3.X
		if self.isv3:
			# detect and extract features from the image
			# descriptor = cv2.xfeatures2d.SIFT_create()
			if self.detector=='sift':
				descriptor = cv2.SIFT_create()
				(kps, features) = descriptor.detectAndCompute(image, None)
			elif self.detector=='orb':
				descriptor = cv2.ORB_create()
				(kps, features) = descriptor.detectAndCompute(image, None)
			elif self.detector=='fast':
				orb = cv2.ORB_create()	
				fast = cv2.FastFeatureDetector_create()
				kps = fast.detect(image)
				(kps, features) = orb.compute(image,kps)
			elif self.detector=='harris':
				gray = np.float32(gray)
				dst = cv2.cornerHarris(gray, blockSize=2, ksize=3, k=0.04)
				# Select and return the keypoints (corners) from the Harris response
				kps = [cv2.KeyPoint(x, y, 1) for y in range(dst.shape[0]) for x in range(dst.shape[1]) if dst[y, x] > 0.01 * dst.max()]
				descriptor = cv2.ORB_create()
				(kps, features) = descriptor.compute(image, kps)
			else:
				print("Unknown detector type: ", self.detector)
				exit(1)
		# otherwise, we are using OpenCV 2.4.X
		else:
			# detect keypoints in the image
			detector = cv2.FeatureDetector_create("SIFT")
			kps = detector.detect(gray)
			# extract features from the image
			extractor = cv2.DescriptorExtractor_create("SIFT")
			(kps, features) = extractor.compute(gray, kps)
		# convert the keypoints from KeyPoint objects to NumPy
		# arrays
		kps = np.float32([kp.pt for kp in kps])
		# return a tuple of keypoints and features
		return (kps, features)
	def matchKeypoints(self, kpsA, kpsB, featuresA, featuresB,
		ratio, reprojThresh):
		# compute the raw matches and initialize the list of actual
		# matches
		matcher = cv2.DescriptorMatcher_create("BruteForce")
		rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
		
		# if self.detector=="fast":
		# 	matcher = cv2.BFMatcher(cv2.NORM_HAMMING,crossCheck=True)
		# 	rawMatches = matcher.match(featuresA,featuresB)
		# 	print(f"Number of raw matches: {len(rawMatches)}")
		# else:
		# 	matcher = cv2.DescriptorMatcher_create("BruteForce")
		# 	rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
		matches = []
		# loop over the raw matches
		for m in rawMatches:
			# ensure the distance is within a certain ratio of each
			# other (i.e. Lowe's ratio test)
			if len(m) == 2 and m[0].distance < m[1].distance * ratio:
				matches.append((m[0].trainIdx, m[0].queryIdx))
		# computing a homography requires at least 4 matches
		if len(matches) > 4:
			# construct the two sets of points
			ptsA = np.float32([kpsA[i] for (_, i) in matches])
			ptsB = np.float32([kpsB[i] for (i, _) in matches])
			# compute the homography between the two sets of points
			(H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC,
				reprojThresh)
			# return the matches along with the homograpy matrix
			# and status of each matched point
			return (matches, H, status)
		# otherwise, no homograpy could be computed
		return None
	def drawMatches(self, imageA, imageB, kpsA, kpsB, matches, status):
		# initialize the output visualization image
		(hA, wA) = imageA.shape[:2]
		(hB, wB) = imageB.shape[:2]
		vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
		vis[0:hA, 0:wA] = imageA
		vis[0:hB, wA:] = imageB
		# loop over the matches
		for ((trainIdx, queryIdx), s) in zip(matches, status):
			# only process the match if the keypoint was successfully
			# matched
			if s == 1:
				# draw the match
				ptA = (int(kpsA[queryIdx][0]), int(kpsA[queryIdx][1]))
				ptB = (int(kpsB[trainIdx][0]) + wA, int(kpsB[trainIdx][1]))
				cv2.line(vis, ptA, ptB, (0, 255, 0), 1)
		# return the visualization
		return vis
def trim(image):
	
	if not np.sum(image[0]):
		return trim(image[1:])
	if not np.sum(image[-1]):
		return trim(image[:-2])
	if not np.sum(image[:,0]):
		return trim(image[:,1:])
	if not np.sum(image[:,-1]):
		return trim(image[:,:-2])
	return image

In [2]:
imageA = cv2.imread("../Data/Left_cs.jpg")
imageB = cv2.imread("../Data/Middle_cs.jpg")
imageC = cv2.imread("../Data/Right_cs.jpg")
imageA = imutils.resize(imageA, width=400)
imageB = imutils.resize(imageB, width=400)
imageC = imutils.resize(imageC, width=400)
# stitch the images together to create a panorama
detector_list = ['sift','orb','fast','harris']

for detector in detector_list:
    stitcher = Stitcher(detector)
    (resultLM,visLM) = stitcher.stitch([imageA, imageB], showMatches=True)
    cv2.imwrite(f"../Image/Paro/{detector}/panorama_keypoints_LM.jpg",visLM)
    (resultMR,visMR) = stitcher.stitch([imageB, imageC], showMatches=True)
    cv2.imwrite(f"../Image/Paro/{detector}/panorama_keypoints_MR.jpg",visMR)
    cv2.imwrite(f"../Image/Paro/{detector}/panorama_resultMR.jpg",resultMR)
    resultLM = trim(resultLM)
    resultMR = trim(resultMR)
    (result, vis) = stitcher.stitch([resultLM,resultMR], showMatches=True)
    result = trim(result)
    cv2.imwrite(f"../Image/Paro/{detector}/panorama.jpg",result)
    cv2.imwrite(f"../Image/Paro/{detector}/panorama_keypoints.jpg",vis)
    print(f"{detector} is successfully done...")


# """
# Counter 1 -> Means -> Results of matching points from left to middle
# Counter 2 -> Means -> Results of matching points from middle to right
# Counter 3 -> Means -> Results of matching points from left-middle to middle-right 
# """

For counter = 1 and detector = sift the matchpoint number is 378
For counter = 2 and detector = sift the matchpoint number is 304
For counter = 3 and detector = sift the matchpoint number is 440
sift is successfully done...
For counter = 1 and detector = orb the matchpoint number is 101
For counter = 2 and detector = orb the matchpoint number is 66
For counter = 3 and detector = orb the matchpoint number is 109
orb is successfully done...
For counter = 1 and detector = fast the matchpoint number is 634
For counter = 2 and detector = fast the matchpoint number is 371
For counter = 3 and detector = fast the matchpoint number is 687
fast is successfully done...
For counter = 1 and detector = harris the matchpoint number is 461
For counter = 2 and detector = harris the matchpoint number is 298
For counter = 3 and detector = harris the matchpoint number is 505
harris is successfully done...
